In [23]:
from collections import deque
from typing import Dict, Optional
from openai import OpenAI
import datetime


In [24]:
client = OpenAI()


In [25]:
def detect_language(text: str) -> str:
    arabic_chars = sum(1 for c in text if '\u0600' <= c <= '\u06FF')
    return "ar" if arabic_chars > 3 else "en"


In [26]:
SYSTEM_PROMPT_EN = """
You are a Jordanian Employment Contract Drafting Assistant.

Rules:
- Draft NON-BINDING employment contracts only.
- Do NOT provide legal advice.
- Use neutral, professional language.
- Ask for missing information before drafting.
- Regenerate the FULL contract on every edit.
- Always include a legal disclaimer at the end.
"""

SYSTEM_PROMPT_AR = """
أنت مساعد لصياغة عقود عمل أردنية لأغراض معلوماتية فقط.

القواعد:
- الصياغة غير ملزمة قانونيًا.
- لا تقدم استشارة قانونية.
- استخدم لغة رسمية ومحايدة.
- اسأل عن أي معلومات ناقصة قبل الصياغة.
- أعد صياغة العقد كاملًا عند أي تعديل.
- أضف تنويهًا قانونيًا إلزاميًا في النهاية.
"""


In [27]:
QUESTION_FLOW = {
    "en": [
        ("contract_type", "What type of contract is this? (limited / unlimited)"),
        ("job_title", "What is the job title?"),
        ("employee_name", "What is the employee’s full name?"),
        ("employer_name", "What is the employer’s name?"),
        ("work_location", "What is the work location?"),
        ("start_date", "What is the start date?"),
        ("end_date", "What is the end date? (skip if unlimited)"),
        ("salary", "What is the monthly salary and currency?"),
        ("working_hours", "What are the daily working hours?"),
        ("working_days", "What are the working days per week?"),
        ("termination", "Briefly describe termination terms.")
    ],
    "ar": [
        ("contract_type", "ما نوع العقد؟ (محدد / غير محدد)"),
        ("job_title", "ما المسمى الوظيفي؟"),
        ("employee_name", "ما اسم الموظف الكامل؟"),
        ("employer_name", "ما اسم صاحب العمل؟"),
        ("work_location", "ما مكان العمل؟"),
        ("start_date", "ما تاريخ بدء العمل؟"),
        ("end_date", "ما تاريخ انتهاء العقد؟ (اتركه فارغًا إذا كان غير محدد)"),
        ("salary", "ما مقدار الراتب الشهري وعملته؟"),
        ("working_hours", "كم عدد ساعات العمل اليومية؟"),
        ("working_days", "ما أيام العمل الأسبوعية؟"),
        ("termination", "اذكر شروط إنهاء العقد.")
    ]
}


In [28]:
chat_state = {
    "language": None,
    "question_index": 0,
    "contract_data": {},
    "draft_versions": deque(maxlen=6),
    "current_draft": None,
    "stage": "collecting"  # collecting → drafted
}


In [29]:
def greet_user(lang: str) -> str:
    if lang == "ar":
        return (
            "مرحبًا 👋\n"
            "أنا مساعد لصياغة عقود العمل في الأردن.\n\n"
            "🔹 أقوم بصياغة عقود **غير ملزمة قانونيًا** لأغراض معلوماتية فقط.\n"
            "🔹 سأطرح عليك بعض الأسئلة خطوة بخطوة.\n\n"
            "لنبدأ: 👇"
        )
    else:
        return (
            "Hello 👋\n"
            "I am a Jordanian employment contract drafting assistant.\n\n"
            "🔹 I generate **non-binding contracts** for informational purposes only.\n"
            "🔹 I will ask you a few questions step by step.\n\n"
            "Let’s begin: 👇"
        )


In [30]:
def generate_contract(data: Dict, lang: str) -> str:
    system_prompt = SYSTEM_PROMPT_AR if lang == "ar" else SYSTEM_PROMPT_EN

    user_prompt = f"""
Draft an employment contract using the following details:

{data}

Include a legal disclaimer at the end.
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    return response.choices[0].message.content.strip()


In [31]:
def contract_chatbot(user_input: str) -> str:
    # First message
    if chat_state["language"] is None:
        chat_state["language"] = detect_language(user_input)
        return greet_user(chat_state["language"])

    lang = chat_state["language"]

    # Collecting data
    if chat_state["stage"] == "collecting":
        key, _ = QUESTION_FLOW[lang][chat_state["question_index"]]
        chat_state["contract_data"][key] = user_input.strip()
        chat_state["question_index"] += 1

        if chat_state["question_index"] < len(QUESTION_FLOW[lang]):
            return QUESTION_FLOW[lang][chat_state["question_index"]][1]

        # All info collected
        draft = generate_contract(chat_state["contract_data"], lang)
        chat_state["draft_versions"].append(draft)
        chat_state["current_draft"] = draft
        chat_state["stage"] = "drafted"
        return draft

    # Editing stage
    if chat_state["stage"] == "drafted":
        updated_data = dict(chat_state["contract_data"])
        updated_data["requested_edit"] = user_input

        new_draft = generate_contract(updated_data, lang)
        chat_state["draft_versions"].append(new_draft)
        chat_state["current_draft"] = new_draft

        return new_draft
